# **Week 1 : Linear Algebra and Calculus**

In this assignment, we shall explore the concepts of analytic and numeric computation of gradients. Further, we will have a look at the computation graph which is a central concept to building a neural network. For learning how gradients are computed analytically, refer to the resources provided in this week.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWsYeD8ZlYxFBB33qIn7bwQvP-KuvLZXJOoA&usqp=CAU"
 style="float:center;width:50px;height:50px;">

# **Importing Libraries**
Feel free to import any additional libraries required

In [1]:
# Import all libraries here

import numpy as np
import random
import math



# Setting the seed for reproducible results


# *Problem 1(a)*

In this problem, we shall be exploring the concepts of analytic and numeric computation of gradients for scalar valued functions. 

\begin{equation}
z = w^{T}x + b \\ 
\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}}\\ 
L(\hat{y}, y) = y.log(\hat{y}) + (1-y).log(1-\hat{y})
\end{equation}

For this set of equations, the vector w, and scalars b, y are to be treated as constants. \\

Now, let us find the analytic gradient of the function L with respect to the function x. That is, compute $\frac{\delta L}{\delta x}$. Write the answer obtained as code in the function provided.

Initialise $w$ to a $10 \times 1$ vector sampled over a standard multivariate gaussian distribution, b to a uniform random variable over the interval $(0, 1)$, y to a bernoulli random variable over $\{0, 1\}$

In [2]:
from numpy.core.fromnumeric import transpose
# Initialisation : 

w = np.random.multivariate_normal(np.zeros(1), np.eye(1), size=10)
b = np.random.uniform(0,1)

rvs = np.array([])
for i in range(0,1):
  if np.random.rand() <= 1:
    a=1
    rvs = np.append(rvs,a)
  else:
    a=0
    rvs = np.append(rvs,a)
y = rvs
dx = 1
x = np.array([1,1,1,1,1,1,1,1,1,1])
w_trans = w.reshape(1,10)
print(w)
print(w_trans)
print(y)
print(b)
print(x)




[[-0.54582872]
 [ 1.02505867]
 [-0.38016878]
 [-0.80132886]
 [-0.11866275]
 [-1.18658396]
 [ 1.63857828]
 [ 0.76753926]
 [ 0.98342385]
 [-1.18689568]]
[[-0.54582872  1.02505867 -0.38016878 -0.80132886 -0.11866275 -1.18658396
   1.63857828  0.76753926  0.98342385 -1.18689568]]
[1.]
0.7233961511845961
[1 1 1 1 1 1 1 1 1 1]


In [ ]:
def analytic_grad(x) :
  for i in range(len(w_trans)):
    ### WRITE CODE BELOW ##
    z = np.dot(w_trans[i],x) + b
    a = math.exp(-z)
    gradient = (1-y)*(-w_trans[i])* a + w_trans[i]/ (1 +  a)
  return gradient
print(analytic_grad(x))

[ 0.00438021  0.00381516 -0.002455   -0.01462927 -0.0026232  -0.00319769
 -0.00321689 -0.00112851 -0.00705618 -0.01020404]


# *Problem 1(b)*

Now, we compute the numeric gradient for the function L. Refer to [this](https://stackoverflow.com/questions/38854363/is-there-any-standard-way-to-calculate-the-numerical-gradient) stack exchange post to see how numeric gradients are computed

In [3]:
from numpy.core.getlimits import log10
def numeric_grad(x) :
  x = np.array([1,1,1,1,1,1,1,1,1,1]) 
  h = 1
  ### WRITE CODE BELOW ###
  def function(a):
    for i in range(len(w_trans)):
      z = np.dot(w_trans[i],x[a]) + b
      
      s = 1/ (1+np.exp(-z))
      fx = y*np.log(s) + np.log(1-s) - y*np.log(1-s)
    return fx
  gradient = (function(i + h) - function(i - h))/2 
  return  gradient
  ### WRITE CODE ABOVE ###
print(numeric_grad(x))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# *Problem 2*

Here, we'll be looking at computational graphs, and their calculus, finding gradients of  variables with respect to other variables in the graph.



We'll be looking at nodes of the graph that are operation based, i.e., each operation performed has a node associated with it.

We'll provide you with example implementations for three of the nodes, the *add* node, the *nth power* node and the *sine* node, you'll have to write the classes for all the other nodes.

# **Multi-input nodes**

In [4]:
class Add: 
  
  # A class to add multiple variables together
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # Scalar addition is taken care of in a separate node, we could have fit into this node but thought it might be easier if it wasn't.
                               # So cases like "a + 1" are to be done separately, and cases like "b + c + d + 4" can be done as "(b + c + d) + 4" or "b + c + (d + 4)", since our scalar addition supports only one variable and one scalar, we'll get to that later
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    return (np.sum(self.lst_values))# This computes the sum of all the variables
  
  def compute_gradients(self):
    return dict(zip(self.lst_names, np.ones(len(self.lst_names)))) # This gives the gradient of the sum wrt to all the input variables, as a dictionary, will be used later


class Multiply: 
  
  #Everything's almost the same as the add class, but this deals with multiplication of more than 1 variables
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # Scalar multiplication is taken care of in a separate node
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.prod(self.lst_values)
  
  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    n = len(self.lst_values)
    left = [0]*n
    right = [0]*n
    prod = [0]*n
    left[0] = 1
    right[n-1] = 1
    for i in range(1, n):
      left[i] = self.lst_values[i - 1] * left[i - 1]
    for j in range(n-2,-1,-1):
      right[j] = self.lst_values[j + 1] * right[j + 1]
  	
    for i in range(n):
      prod[i] = left[i] * right[i]
	  

    return dict(zip(self.lst_names, prod))
obj = Multiply(['a','b','c'],[1,2,3])
print(Multiply.compute_output)

<function Multiply.compute_output at 0x7f5948627280>


# **Scalar multiplication/addition nodes**

In [5]:
class Add_Scalar: 
  
  # A class to add a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be added.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    sum =  np.sum(self.lst_values)
    for i in range(len(self.lst_names)):
      if self.lst_names[i].isnumeric() == True:
        j = int(self.lst_names[i])
        sum += j
    return sum


  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    return dict(zip(self.lst_names, np.ones(1)))



class Multiply_Scalar: 
  
  # A class to multiply a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be multiplied.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    product =  np.prod(self.lst_values)
    for i in range(len(self.lst_names)):
      if self.lst_names[i].isnumeric() == True:
        j = int(self.lst_names[i])
        product = j * product
    return product

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
     return dict(zip(self.lst_names, self.lst_names[1]))


# **Nodes for special functions**

In [6]:
class Power:

  def __init__(self, name, value, exponent):
    self.name = name # Name of the variable to be exponentiated
    self.value = value # Value of the variable
    self.exponent = exponent # Value of the exponent
  
  def compute_output(self):
    return np.power(self.value, self.exponent)
  
  def compute_gradients(self):
    return {self.name : self.exponent*(np.power(self.value, (self.exponent - 1)))}

class Sine: 

  # A class to apply the sine function on a variable
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.sin(self.value)
  
  def compute_gradients(self):
    return {self.name : np.cos(self.value)}


class Logarithm:

  # A class to compute the logarithm of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.log(self.value)

  def compute_gradients(self):
    # Write your code here
    j = 1/np.log(self.value)
    return {self.name : j}


class Exponential: 

  # A class to compute the exponential of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.exp(self.value)

  def compute_gradients(self):
    # Write your code here
    exp = np.exp(self.value)
    grad = self.value * exp
    return {self.name :grad}




Now that we have these classes, let's use them to actually find gradients of complex networks. We finally bring in the idea of a computational graph, which makes it much easier for the gradients to be computed.

This is the image of the example graph that we want you to work with. 





<div>
<img src="https://drive.google.com/uc?id=1VtI1lf85bG8cO1u_8l0D0rqVGhwHQtPK"
 width="500" height="500">
</div>

\begin{equation}
d = 3a \\ 
e = abc \\ 
f = sin(c) \\ 
g = exp(e) \\ 
h = a + d + g + f
\end{equation}

# **Forward Propogation**

In [15]:
def forward_prop(a, b, c) : 
    '''
    Input : real numbers a, b, c.

    Outputs : A dictionary of the values at each node with keys as the names of nodes
    Grads : A dictionary of the gradients at each edge with keys as a pair of nodes 
    
    e.g. Grads["ce"] = ...
    '''
    d_obj = Multiply_Scalar(['a','3'],[3])
    d = d_obj.compute_output
    ad = d_obj.compute_gradients
    

    e_obj = Multiply(['a','b','c'],[a,b,c])
    e = e_obj.compute_output
    ce = e_obj.compute_gradients

    f_obj = Sine(['c'],[c])
    f = f_obj.compute_output
    cf = f_obj.compute_gradients

    g_obj = Exponential(['e'],[e])
    g = g_obj.compute_output
    eg = g_obj.compute_gradients

    h_obj = Add(['a','d','g','f'],[a,d,g,f])
    h  = h_obj.compute_output
    dh = h_obj.compute_gradients

    out = np.array([d,e,f,g,h])
    names_out = ['ad','ce','cf','eg','dh']
    gradient = np.array([ad,ce,cf,eg,dh])
    
    Outputs = dict(zip(names_out, out))
    Grads = dict(zip(names_out, gradient))


    return (Outputs, Grads)
print(forward_prop(1,2,3))

<bound method Multiply_Scalar.compute_output of <__main__.Multiply_Scalar object at 0x7f594406e160>> <bound method Multiply_Scalar.compute_gradients of <__main__.Multiply_Scalar object at 0x7f594406e160>>
({'ad': <bound method Multiply_Scalar.compute_output of <__main__.Multiply_Scalar object at 0x7f594406e160>>, 'ce': <bound method Multiply.compute_output of <__main__.Multiply object at 0x7f594406e190>>, 'cf': <bound method Sine.compute_output of <__main__.Sine object at 0x7f594406e2b0>>, 'eg': <bound method Exponential.compute_output of <__main__.Exponential object at 0x7f594406e310>>, 'dh': <bound method Add.compute_output of <__main__.Add object at 0x7f594406e370>>}, {'ad': <bound method Multiply_Scalar.compute_gradients of <__main__.Multiply_Scalar object at 0x7f594406e160>>, 'ce': <bound method Multiply.compute_gradients of <__main__.Multiply object at 0x7f594406e190>>, 'cf': <bound method Sine.compute_gradients of <__main__.Sine object at 0x7f594406e2b0>>, 'eg': <bound method Ex

# **Backward Propogation**

Most of the usage of computational graphs in Machine Learning centers around finding the gradients of a particular loss, wrt to all the parameters. Here, your task is to do a simpler version of that.

Use the classes you wrote to calculate the following gradients : 


*   $ \frac{\partial h}{\partial a}$
*   $ \frac{\partial h}{\partial b}$
*   $ \frac{\partial h}{\partial c}$

Use the technique of *backpropogation* to code out the gradients step-wise, along all possible chains of the graph starting from $h$ and ending at $a,b,c$ respectively. 

Don't try to directly get these values without backpropogation, it might be easier here, but with complicated neural networks it wouldn't be :) 



In [8]:
def backward_prop(a, b, c, Outputs, Grads) : 
    '''
    Input : a, b, c (input layer); Outputs (values at each node); Grads (gradients stored at each edge)
    Retuns : result (gradients w.r.t a, b, c)
    '''
    dh = list(Grads.values())[4]
    a = list(dh.values())[0]
    dd = list(Grads.values())[0]
    d = list(dd.values())[0]
    dg = list(Grads.values())[3]
    e2 = list(dg.values())[0]
    g = list(dg.values())[0]

    dh_da = a + d + g*e2

    dg = list(Grads.values())[3]
    g = list(dg.values())[0]
    de = list(Grads.values())[2]
    e = list(dg.values())[1]

    dh_db = g * e

    df = list(Grads.values())[2]
    f = list(df.values())[0]
    e1 = list(dg.values())[2]

    dh_dc = g*e1 + f
    result = [dh_da, dh_db, dh_dc]



    
    return result

# **Combining both processes**

For the purpose of values, assume that $a = 3, b = 1, c = 2$. Here, we call both forward and backward propogation functions to demonstrate functionality of the functions above.

In [14]:
# Initialisation
a = 3
b = 1
c = 2

# Forward propogation
(Outputs, Grads) = forward_prop(a, b, c)
print(tuple(Outputs), Grads)
print(f'Value obtained upon forward propogation : {Output['h']}')


# Backward propogation
result = backward_prop(a, b, c, Outputs, Grads)
print(f'Values of gradients are : {result[0]}, {result[1]}, {result[2]}')

SyntaxError: ignored

# **Submission Instructions**

Upload this notebook on your github classroom repository by the name Week1.ipynb